# Look at the big picture.


## Libraries

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from statistics import median

In [13]:
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [14]:
# Activamos las alertas de estilo
%pycodestyle_on

## Performance Metric

In [2]:
def performance(y_true: list, y_pred: list) -> float:
    """

    """

    CF = [1 for _ in y_true if y_pred > y_true] / len(y_pred) * 100
    RMSE = mean_squared_errors(y_true, y_pred)
    sEMSE = RMSE / median(y_true)
    metric = (0.7 * rRMSE) + (0.3 * (1 - CF))

    return metric

# Get the data

In [142]:
# Read the data
data = pd.read_csv('./data/Modelar_UH2021.txt', delimiter="|")

## Overview the data

In [143]:
data.head()

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
0,1/6/2015 0:00:00,21972,0,C,75.0,No Rotura,NaN,0,0,5241.0,0
1,1/6/2015 0:00:00,23910,5,C,170.0,No Rotura,"6,07",0,0,5241.0,3
2,1/6/2015 0:00:00,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
3,1/6/2015 0:00:00,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
4,1/6/2015 0:00:00,27144,15,E,230.0,No Rotura,NaN,0,0,4064.0,0


In [5]:
data.describe()

,id,visitas,categoria_dos,dia_atipico,campaña,antiguedad,unidades_vendidas
count,4.045022e+06,4.045022e+06,4.039178e+06,4.045022e+06,4.045022e+06,3.170857e+06,4.045022e+06
mean,2.745180e+05,1.721371e+02,2.016524e+02,6.365503e-02,2.801715e-03,1.011114e+03,4.693434e+00
std,8.336719e+04,6.889116e+02,1.179997e+02,3.597475e-01,5.285704e-02,7.162509e+02,2.237403e+01
min,2.197200e+04,0.000000e+00,2.000000e+00,-1.000000e+00,0.000000e+00,1.260000e+02,0.000000e+00
25%,2.122940e+05,7.000000e+00,8.200000e+01,0.000000e+00,0.000000e+00,5.240000e+02,0.000000e+00
50%,2.926720e+05,3.500000e+01,2.360000e+02,0.000000e+00,0.000000e+00,7.950000e+02,0.000000e+00
75%,3.379000e+05,1.300000e+02,3.070000e+02,0.000000e+00,0.000000e+00,1.244000e+03,3.000000e+00
max,4.586600e+05,1.200450e+05,3.430000e+02,1.000000e+00,1.000000e+00,5.310000e+03,4.881000e+03


In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4045022 entries, 0 to 4045021
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   fecha              object 
 1   id                 int64  
 2   visitas            int64  
 3   categoria_uno      object 
 4   categoria_dos      float64
 5   estado             object 
 6   precio             object 
 7   dia_atipico        int64  
 8   campaña            int64  
 9   antiguedad         float64
 10  unidades_vendidas  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 339.5+ MB


# Discover and visualize the data to gain insights

# Prepare the data for Machine Learning algorithms

In [147]:
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime

class time_format(BaseEstimator, TransformerMixin):
    
    def __init__(self, document='Modelar'):
        self.document = document
        
    def fit(self, X, y=None):
        return self  # nothing else to do    
        
    def transform(self, data):
        """
        Take the dataframe and trate the 'fecha' attribute depending of the document.
        If it is the 'Modelar' document, the variable will be equal to the content 
        before the first space. 
        If it it the 'Estimar' document, the variable will be transformed to the 
        original format 

        Original Format: 'DD/MM/AAAA' in (text format)


        """  
        temp = ''

        if self.document == 'Modelar':
            data['fecha'] = pd.to_datetime(data['fecha'], infer_datetime_format=True).apply(lambda x : x.strftime('%d/%m/%Y'))
        elif self.document == 'Estimar':
            data['fecha']    # agregar la modificación para el documento Estimar
        else:
            print('Unknown document!!!')

    
        return data

In [148]:
from sklearn.pipeline import Pipeline

data_prep_pipeline = Pipeline([
         ('time_format', time_format(document='Modelar'))
])

data_prep_pipeline.fit_transform(data)

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
0,06/01/2015,21972,0,C,75.0,No Rotura,NaN,0,0,5241.0,0
1,06/01/2015,23910,5,C,170.0,No Rotura,"6,07",0,0,5241.0,3
2,06/01/2015,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
3,06/01/2015,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
4,06/01/2015,27144,15,E,230.0,No Rotura,NaN,0,0,4064.0,0
...,...,...,...,...,...,...,...,...,...,...,...
4045017,30/09/2016,457416,1395,F,336.0,No Rotura,"50,38",0,0,130.0,9
4045018,30/09/2016,457422,1080,F,336.0,Rotura,NaN,0,0,130.0,0
4045019,30/09/2016,458650,1385,K,340.0,No Rotura,NaN,0,0,126.0,0
4045020,30/09/2016,458660,1915,K,340.0,No Rotura,"68,49",0,0,126.0,24


In [124]:
data.head()

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
0,06/01/2015,21972,0,C,75.0,No Rotura,NaN,0,0,5241.0,0
1,06/01/2015,23910,5,C,170.0,No Rotura,"6,07",0,0,5241.0,3
2,06/01/2015,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
3,06/01/2015,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
4,06/01/2015,27144,15,E,230.0,No Rotura,NaN,0,0,4064.0,0


# Select a model and train it

# Fine-tune your model

# Present your solution

## Carga de los datos

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
0,1/6/2015 0:00:00,21972,0,C,75.0,No Rotura,NaN,0,0,5241.0,0
1,1/6/2015 0:00:00,23910,5,C,170.0,No Rotura,"6,07",0,0,5241.0,3
2,1/6/2015 0:00:00,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
3,1/6/2015 0:00:00,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
4,1/6/2015 0:00:00,27144,15,E,230.0,No Rotura,NaN,0,0,4064.0,0


## Data Cleaning